In [6]:
# librairies
from sgp4.earth_gravity import wgs72
from sgp4.io import twoline2rv
import math
%pylab 
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
import os
import re
import subprocess
from time import sleep
from sklearn.neighbors import KNeighborsRegressor

def time_to_LB(): # main function. It's agregation rusults of over functions, making tables and graphics.
    global satellite_name
    dates=date_generator() # obtain dates
    if input("Convert dates to UTC?(y/n)")=='y':
        dates=data.index.tz_localize('Europe/Moscow').tz_convert('utc')
    file_name=r'import data/'+input('Type the TLE filename(e.g. "glo-ops.txt", note: file must be in .txt extension, \n and plased in folder "import data"):  ') # type name of the file with TLE data
    satellite_name=input('Type satellite name(e.g. "COSMOS 2456"):  ') # previous file should contain TLE data for specifyed satellite
    with open(str(file_name), 'r') as TLE_data:
        all_data=TLE_data.readlines()
    for pos,line in enumerate(all_data):
        if line.find(satellite_name)!=-1:
            TLE_first_str=all_data[pos+1].strip() # read first line of TLE data
            TLE_sec_str=all_data[pos+2].strip()# # read second line of TLE data
            break
        elif pos==len(all_data): # if we finished end of the file and didn't find anythyng
            print('Couldn\'t find TLE')
    coord=[] # list for coordinates
    print("Running program: ")
    for procent,date in enumerate(dates): #procent using for progress bar
        print("\r{}%".format(int(ceil(procent*100/len(dates)))), end="") # progress bar
        #sys.stdout.flush() #<- makes python print it anyway, sometimes program shut down because of this line, so I commented her
        coord.append(state_vector(TLE_first_str, TLE_sec_str, date)) # obtain state vector for specifyed TLE and date
    coord= pd.DataFrame(coord, columns=['year','mon','day','hour','min','sec', 'x','y','z','altitude',  'latitude', 'longitude', "longitude_"]) # transform list into more convenient DataFrame
    coord_for_igrf=coord.drop(['mon','day','hour','min','sec','x','y','z'], axis=1) # make pivot table for external programm that calculate L,B coordinates
    try:#try to delete temporal files, they are using by igrf.exe, if they doesn't exist it's OK
        os.remove('Path.txt')
        os.remove('Result.txt')
    except FileNotFoundError:
        print('')
    coord_for_igrf.to_csv('Path.txt', sep=' ', index=False,header=False) #make file with prepaired pivot table
    sleep(1)
    subprocess.run('igrf.exe') # this programm reads by defaul file 'Path.txt', so they must be in one folder. When finished, it creates resulting file "Result.txt". NOTE! igrf.exe won't create result file if there is one with the same name
    sleep(1)
    igrf_res=pd.read_csv('Result.txt', delim_whitespace=True)
    sleep(1)
    coord['L']=igrf_res['L-VALUE'] # append obtained l and B coordinates
    coord['B/B0']=igrf_res['B/B0']
    coord['epoch']=dates
    if len(dates)==1: # if in the beginning you entered just one date, then program prints it out on the screen
        print('\nRequested data by {}: \n \t state vector: x:{}, y:{}, z:{})\n \t longitude: {}\n \t latitude: {} \n \t L, B/B0: {}, {}'.format(str(coord[['year','mon','day','hour','min','sec']].ix[0].values),coord.x.ix[0],coord.y.ix[0],coord.z.ix[0],coord.longitude.ix[0],coord.latitude.ix[0], coord.L.ix[0],coord['B/B0'].ix[0]))
    if input('\nsave satellite path to file? y/n: ')=='y': # try to save obtained data to .csv file wth separator ';'
        while True:
            try:
                path=input('Type the file name: ')
                coord.to_csv(r'export data/%s.csv' % path, header=coord.columns, index=False)
                break
            except OSError:
                print('Name shouldn\'t contain \|/:*"<>?' )
    if input('Plot picture of the satellite %s sub-path? y/n: '% satellite_name)=='y': # try to plot sub-path on the Earth map in Miller projection  
        plt.close('all')
        fig, ax=plt.subplots(nrows=1,ncols=1,figsize=(23.9,12.5))
        m=Basemap(ax=ax)
        m.drawcoastlines()
        m.drawcountries()
        #m.etopo()
        x,y=m(coord.longitude,coord.latitude)
        if len(data)!=0 and len(data.columns)!=0:
            clf=KNeighborsRegressor(n_neighbors=40, weights='uniform')
            data['date_coded']=[x.value for x in data.index]
            pred=pd.DataFrame(clf.fit(data['date_coded'].dropna().reshape(-1,1),data[4].dropna()).predict(data['date_coded'].dropna().reshape(-1,1)))
            pred=pd.DataFrame(clf.predict(data['date_coded'].reshape(-1,1)))
            pred.index=data.index
            print(pred)
            pred.plot()
            b=list(pred[0])
            b.append(pred[0][len(pred)-1])
            b.pop(0)
            d=np.array(b)-pred[0].values
            path=m.scatter(x,y, c=d/d.mean(),marker=6,s=600, alpha=1)
            clb=fig.colorbar(path, ax=ax, shrink=0.761)
            clb.set_label('*{},Rad'.format(round(pred[0].mean(),2)))
        m.plot(x,y,'r+')
        ax.set_title('The satellite {} sub-path for the dates {}-{}'.format(satellite_name, start_date, end_date))
        if input('Save obtained picture as png? y/n: ')=='y':
            plt.savefig(r'export data/The satellite {} sub-path for the dates {}--{}.png'.format(satellite_name, str(data.index[0]).replace(':','-'), str(data.index[-1]).replace(':','-')))
    
def date_generator():
    global data
    global start_date 
    global end_date
    data=pd.DataFrame()
    dates=[]
    while True:
        dates_from_file=input('Dates from file? y/n: ')
        if dates_from_file=='y':
            while True:
                try:
                    path=r'import data/'+input('Type filename(e.g. "dates.csv"(without quotes); note: separator=",",decimal=".",dayfirst=True, dates must be in UTC, \n and plased in time_to_LB.ipynb directory in folder "input data"): ')
                    print("Loading file:")
                    data= pd.read_csv(path, sep=',',decimal='.', header=None, parse_dates=[0],dayfirst=False, infer_datetime_format=True, index_col=0)
                    print("Succeed!")
                    break
                except OSError:
                    print('Invalid filename, try again')
            if input('All dates from file? y/n: ')=='n':
                start_date=input('Input the date of the beginning in the format: "YYYY-MM-DD hh:mm:ss":  ')
                end_date=input('Input the date of the end in the format: "YYYY-MM-DD hh:mm:ss":  ')
                data=data[start_date:end_date]
            else:
                start_date=data.index[0]
                end_date=data.index[-1]
            print(data)
            return data.index
            break
        elif dates_from_file=='n':
            date_or_range=input('Certain date or range? y - certain date, n - range: \n')
            if date_or_range=='n':
                start_date=input('Input the date of the beginning in the format: "Y/M/D h:m:s":  ')
                end_date=input('Input the date of the end in the format: "Y/M/D h:m:s":  ')
                freq=input('Input the frequensy in the format: %digit%string % [1,2,3...],["s","min","h","d","m","y"]\n(e.g. "10min" or "1s"): ' )
                dates=pd.date_range(start_date,end_date,freq=freq)
                return dates
                break
            elif date_or_range=='y':
                if input('current date?(Important! your local time will be recalculated into UTC time) y - current date, n - other date: \n')=='y':
                    dates=[datetime.utcnow()]
                    return dates
                    break
                dates=[parse(input('Input the date in the format: "Y/M/D h:m:s":  '))]
                return dates
                break
            else:
                print('Wrong input, try again')
                continue
        else:
            print('Wrong input, try again')
            continue
def star_time(date):
    #d_s=[int(x.strip()) for x in date.split(sep=',')]
    year=date.year#d_s[0]
    mon=date.month#d_s[1]
    day=date.day#d_s[2]
    hour=date.hour#d_s[3]
    minute=date.minute#d_s[4]
    sec=date.second#d_s[5]
    a=math.trunc((14-mon)/12)
    y=year+4800-a
    m=mon+12*a-3
    JDN=day+math.trunc((153*m+2)/5)+365*y+math.trunc(y/4)-math.trunc(y/100)+math.trunc(y/400)-32045
    JD=JDN+(hour-12)/24+minute/1440+sec/86400
    MJD=JD-2400000.5 # modifyed Julian date
    # star time 
    MJD0=math.trunc(MJD)
    UT=(MJD-MJD0)*24
    T=(MJD0-51544.5)/36525.0
    GMST=6.697374558+1.0027379093*UT+(8640184.812866+(0.093104-(6.2e-6*T))*T)*T/3600.0
    def FRAC(X):
        X=X-math.trunc(X)
        if X<0:
            X+=1
        return X
    LMST=24.0*FRAC(GMST/24.0)
    #hour_LMST=math.trunc(LMST)
    #minute_LMST=math.trunc((LMST-math.trunc(LMST))*60)
    #sec_LMST=math.trunc(((LMST-math.trunc(LMST))*60-math.trunc((LMST-math.trunc(LMST))*60))*60)
    return LMST
def state_vector(TLE_first_str, TLE_sec_str, date):
    '''
    This function recuires imported sgp4.earth_gravity.wgs72 and sgp4.io.twoline2rv.
    It brings state vector(coordinates in ECI(in km), and velocity for each axe(in km/h)) for required satellite
    
    Inputs: (file_name, sat_name, data)
            file_name - name of text doc in .txt, where are all TLE data agregated, file_name should ends with '.txt', ex:'glo-ops.txt'
            sat_name - satellite name, specifyed in doc file_name, ex.: 'COSMOS 2419' in file 'glo-ops.txt'
            date - date in string, format: 'YYYY, MM, DD, HH, mm, SS', if didgit of the day, hour, etc. can be written by one digit, then you should write it only in this way, without zeros, ex.:'2016, 9, 29, 13, 56, 59', here month - '9'. NOT '09'!!! 
    Outputs: function returns tuple with state vector and prints extracted TLE data, and date, specifyed in input
    '''
    year=date.year#d_s[0]
    mon=date.month#d_s[1]
    day=date.day#d_s[2]
    hour=date.hour#d_s[3]
    minute=date.minute#d_s[4]
    sec=date.second#d_s[5]
    # obtain state vector in ECI:
    satellite=twoline2rv(TLE_first_str, TLE_sec_str, wgs72)
    position,velocity=satellite.propagate(year, mon, day, hour, minute, sec) 
    x=position[0]
    y=position[1]
    z=position[2]
    # transit from ECI to longitude, latitude in inertial coordiantes
    longitude=math.atan(y/x)*180/math.pi
    sign=lambda x: int(math.copysign(1,x))
    if (sign(y)>0 and sign(x)<0) or (sign(y)<0 and sign(x)<0):
        longitude+=180
    if sign(y)<0 and sign(x)>0:
        longitude+=360
    r=math.sqrt(x**2+y**2)
    lat=math.atan(z/r)
    # calculate star time for Gregorian calendar
    LMST=star_time(date)
    delta_longitude=7.2921158553*10**(-5)*LMST*3600*180/math.pi
    # transit from polar inetial system to polar relative system
    long_rel=longitude-delta_longitude
    long_rel_=long_rel
    if long_rel>180:
        long_rel-=360
    elif long_rel<-180:
        long_rel+=360
    #recalculate lat, alt for nonspherical Earth:
    a=6378.135 #equatorial Earth's radius
    f=1/298.26 #Earth's shrinkage
    ecc=2*f-pow(f,2) #square of Earth's eccentricity
    while True:
        lat_=lat
        C=1/sqrt(1-ecc*pow(sin(lat_),2))
        lat=math.atan((z+a*C*ecc*math.sin(lat_))/math.sqrt(x**2+y**2))
        if abs(lat_-lat)<0.0001:
            break
    alt=math.sqrt(x**2+y**2)/cos(lat)-a*C
    lat=lat*180/math.pi
    return year, mon, day,hour,minute,sec, x,y,z, alt, lat, long_rel, long_rel_


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\Kotelnikov\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Пример выболнения программы time_to_LB
**Dates from file? y/n:** y 

**Type filename(e.g. "dates.csv"(without quotes); note: separator, 
 and plased in time_to_LB.ipynb directory):** sf
 
**Loading file:
Invalid filename, try again**

**Type filename(e.g. "dates.csv"(without quotes); note: separator, 
 and plased in time_to_LB.ipynb directory):** 11.01.2009  13-19-00--19.05.2009  7-21-00.csv
 
**Loading file:
Succeed!**

**All dates from file? y/n:** n

**Input the date of the beginning in the format: "Y/M/D h:m:s":**  2009/1/11 12:00

**Input the date of the end in the format: "Y/M/D h:m:s":**  2009/1/12 14:56

**Type the TLE filename(e.g. "glo-ops.txt", note: file must be in .txt extension, 
 and plased in time_to_LB.ipynb directory):**  glo-ops.txt
 
**Type satellite name(e.g. "COSMOS 2456"):**  COSMOS 2456

**Running program: 
100%**

**save satellite path to file? y/n:** y

**Type the file name:**COSMOS 2456 12-0-0--11-1-2009 14-56--12-1-2009

**Plot picture of the satellite COSMOS 2456 sub-path? y/n:** y

**Save obtained picture as png? y/n:** n

# time_to_LB 

In [ ]:

time_to_LB()


# Программа для оперирования с большим количеством файлов

In [1]:
# librairies
from sgp4.earth_gravity import wgs72
from sgp4.io import twoline2rv
import math
import re
%pylab 
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
import os
import subprocess
from time import sleep
from sklearn.neighbors import KNeighborsRegressor

def time_to_LB_for_list(file_name,convert_to_UTC,TLE_file_name, sattelite_name): # main function. It's agregation rusults of over functions, making tables and graphics.
    global satellite_name
    global file
    file=file_name
    dates=date_generator() # obtain dates
    if convert_to_UTC:
        dates=data.index.tz_localize('Europe/Moscow').tz_convert('utc')
    file_name=r'import data/'+TLE_file_name#input('Type the TLE filename(e.g. "glo-ops.txt", note: file must be in .txt extension, \n and plased in folder "import data"):  ') # type name of the file with TLE data
    satellite_name=sattelite_name#input('Type satellite name(e.g. "COSMOS 2456"):  ') # previous file should contain TLE data for specifyed satellite
    with open(str(file_name), 'r') as TLE_data:
        all_data=TLE_data.readlines()
    for pos,line in enumerate(all_data):
        if line.find(satellite_name)!=-1:
            TLE_first_str=all_data[pos+1].strip() # read first line of TLE data
            TLE_sec_str=all_data[pos+2].strip()# # read second line of TLE data
            break
        elif pos==len(all_data): # if we finished end of the file and didn't find anythyng
            print('Couldn\'t find TLE')
    coord=[] # list for coordinates
    print("Running program: ")
    for procent,date in enumerate(dates): #procent using for progress bar
        print("\r{}%".format(int(ceil(procent*100/len(dates)))), end="") # progress bar
        #sys.stdout.flush() #<- makes python print it anyway, sometimes program shut down because of this line, so I commented her
        coord.append(state_vector(TLE_first_str, TLE_sec_str, date)) # obtain state vector for specifyed TLE and date
    coord= pd.DataFrame(coord, columns=['year','mon','day','hour','min','sec', 'x','y','z','altitude',  'latitude', 'longitude', "longitude_"]) # transform list into more convenient DataFrame
    coord_for_igrf=coord.drop(['mon','day','hour','min','sec','x','y','z'], axis=1) # make pivot table for external programm that calculate L,B coordinates
    try:#try to delete temporal files, they are using by igrf.exe, if they doesn't exist it's OK
        os.remove('Path.txt')
        os.remove('Result.txt')
    except FileNotFoundError:
        print('')
    coord_for_igrf.to_csv('Path.txt', sep=' ', index=False,header=False) #make file with prepaired pivot table
    sleep(1)
    subprocess.run('igrf.exe') # this programm reads by defaul file 'Path.txt', so they must be in one folder. When finished, it creates resulting file "Result.txt". NOTE! igrf.exe won't create result file if there is one with the same name
    sleep(1)
    igrf_res=pd.read_csv('Result.txt', delim_whitespace=True)
    sleep(1)
    coord['L']=igrf_res['L-VALUE'] # append obtained l and B coordinates
    coord['B/B0']=igrf_res['B/B0']
    coord['epoch']=dates
    if len(dates)==1: # if in the beginning you entered just one date, then program prints it out on the screen
        print('\nRequested data by {}: \n \t state vector: x:{}, y:{}, z:{})\n \t longitude: {}\n \t latitude: {} \n \t L, B/B0: {}, {}'.format(str(coord[['year','mon','day','hour','min','sec']].ix[0].values),coord.x.ix[0],coord.y.ix[0],coord.z.ix[0],coord.longitude.ix[0],coord.latitude.ix[0], coord.L.ix[0],coord['B/B0'].ix[0]))
    if True:#input('\nsave satellite path to file? y/n: ')=='y': # try to save obtained data to .csv file wth separator ';'
        while True:
            try:
                #path=input('Type the file name: ')
                m=re.findall('.+\.', file)
                n=re.sub('\.','',m[0])
                coord.to_excel(r'export data/{}.xlsx'.format(n+'_output'), header=coord.columns, index=False)
                break
            except OSError:
                print('Name shouldn\'t contain \|/:*"<>?' )

def date_generator():
    global data
    global start_date 
    global end_date
    data=pd.DataFrame()
    dates=[]
    while True:
        try:
            path=r'import data/'+file#input('Type filename(e.g. "dates.csv"(without quotes); note: separator=",",decimal=".", dates must be in UTC, \n and plased in time_to_LB.ipynb directory in folder "input data"): ')
            print("Loading file:")
            data= pd.read_csv(path, sep=',',decimal='.', header=None, parse_dates=[0],dayfirst=False, infer_datetime_format=True, index_col=0)
            print("Succeed!")
            return data.index
            break
        except OSError:
            print('Invalid filename, try again')
def star_time(date):
    #d_s=[int(x.strip()) for x in date.split(sep=',')]
    year=date.year#d_s[0]
    mon=date.month#d_s[1]
    day=date.day#d_s[2]
    hour=date.hour#d_s[3]
    minute=date.minute#d_s[4]
    sec=date.second#d_s[5]
    a=math.trunc((14-mon)/12)
    y=year+4800-a
    m=mon+12*a-3
    JDN=day+math.trunc((153*m+2)/5)+365*y+math.trunc(y/4)-math.trunc(y/100)+math.trunc(y/400)-32045
    JD=JDN+(hour-12)/24+minute/1440+sec/86400
    MJD=JD-2400000.5 # modifyed Julian date
    # star time 
    MJD0=math.trunc(MJD)
    UT=(MJD-MJD0)*24
    T=(MJD0-51544.5)/36525.0
    GMST=6.697374558+1.0027379093*UT+(8640184.812866+(0.093104-(6.2e-6*T))*T)*T/3600.0
    def FRAC(X):
        X=X-math.trunc(X)
        if X<0:
            X+=1
        return X
    LMST=24.0*FRAC(GMST/24.0)
    #hour_LMST=math.trunc(LMST)
    #minute_LMST=math.trunc((LMST-math.trunc(LMST))*60)
    #sec_LMST=math.trunc(((LMST-math.trunc(LMST))*60-math.trunc((LMST-math.trunc(LMST))*60))*60)
    return LMST
def state_vector(TLE_first_str, TLE_sec_str, date):
    '''
    This function recuires imported sgp4.earth_gravity.wgs72 and sgp4.io.twoline2rv.
    It brings state vector(coordinates in ECI(in km), and velocity for each axe(in km/h)) for required satellite
    
    Inputs: (file_name, sat_name, data)
            file_name - name of text doc in .txt, where are all TLE data agregated, file_name should ends with '.txt', ex:'glo-ops.txt'
            sat_name - satellite name, specifyed in doc file_name, ex.: 'COSMOS 2419' in file 'glo-ops.txt'
            date - date in string, format: 'YYYY, MM, DD, HH, mm, SS', if didgit of the day, hour, etc. can be written by one digit, then you should write it only in this way, without zeros, ex.:'2016, 9, 29, 13, 56, 59', here month - '9'. NOT '09'!!! 
    Outputs: function returns tuple with state vector and prints extracted TLE data, and date, specifyed in input
    '''
    year=date.year#d_s[0]
    mon=date.month#d_s[1]
    day=date.day#d_s[2]
    hour=date.hour#d_s[3]
    minute=date.minute#d_s[4]
    sec=date.second#d_s[5]
    # obtain state vector in ECI:
    satellite=twoline2rv(TLE_first_str, TLE_sec_str, wgs72)
    position,velocity=satellite.propagate(year, mon, day, hour, minute, sec) 
    x=position[0]
    y=position[1]
    z=position[2]
    # transit from ECI to longitude, latitude in inertial coordiantes
    longitude=math.atan(y/x)*180/math.pi
    sign=lambda x: int(math.copysign(1,x))
    if (sign(y)>0 and sign(x)<0) or (sign(y)<0 and sign(x)<0):
        longitude+=180
    if sign(y)<0 and sign(x)>0:
        longitude+=360
    r=math.sqrt(x**2+y**2)
    lat=math.atan(z/r)
    # calculate star time for Gregorian calendar
    LMST=star_time(date)
    delta_longitude=7.2921158553*10**(-5)*LMST*3600*180/math.pi
    # transit from polar inetial system to polar relative system
    long_rel=longitude-delta_longitude
    long_rel_=long_rel
    if long_rel>180:
        long_rel-=360
    elif long_rel<-180:
        long_rel+=360
    #recalculate lat, alt for nonspherical Earth:
    a=6378.135 #equatorial Earth's radius
    f=1/298.26 #Earth's shrinkage
    ecc=2*f-pow(f,2) #square of Earth's eccentricity
    while True:
        lat_=lat
        C=1/sqrt(1-ecc*pow(sin(lat_),2))
        lat=math.atan((z+a*C*ecc*math.sin(lat_))/math.sqrt(x**2+y**2))
        if abs(lat_-lat)<0.0001:
            break
    alt=math.sqrt(x**2+y**2)/cos(lat)-a*C
    lat=lat*180/math.pi
    return year, mon, day,hour,minute,sec, x,y,z, alt, lat, long_rel, long_rel_


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# time_to_LB_for_list

In [5]:

for name in os.listdir('import data/1'):
    time_to_LB_for_list(name, False, 'METEOR-M 1.txt', 'METEOR-M 1')

Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading file:
Succeed!
Running program: 
100%Loading fi

# Программа для рисования траекторий( все остальные тоже могут рисовать, но здесь какие-то улучшения, не помню какие)

In [67]:
# librairies
from sgp4.earth_gravity import wgs72
from sgp4.io import twoline2rv
import math
%pylab 
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
import os
import subprocess
from time import sleep
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.cm as cm

def time_to_LB_picture_plotting(): # main function. It's agregation rusults of over functions, making tables and graphics.
    global satellite_name
    global data
    global start_date 
    global end_date
    data=pd.DataFrame([[1,2],[1,1]])
    dates_all=date_generator() # obtain dates
    #print(data)
    step=int(input('Type sampling step:\n'))
    start_date=0
    end_date=5120
    dates=[]
    clf=KNeighborsRegressor(n_neighbors=40, weights='uniform')
    data['date_coded']=[x.value for x in data.index]
    pred=pd.DataFrame(clf.fit(data['date_coded'].dropna().reshape(-1,1),data[4].dropna()).predict(data['date_coded'].dropna().reshape(-1,1)))
    pred=pd.DataFrame(clf.predict(data['date_coded'].reshape(-1,1)))
    pred.index=data.index
    '''pred.plot() #If data about cosmic enviroment is absorbet doese then enable 
    b=list(pred[0])
    b.append(pred[0][len(pred)-1])
    b.pop(0)
    d=np.array(b)-pred[0].values
    d=d/d.max()
    d_colored=cm.jet(d)'''
    d=pred[0].values/pred[0].values.max()
    d_colored=cm.jet(d)
    m_ = cm.ScalarMappable(cmap=cm.jet)
    while len(dates)<len(dates_all):
        #input('')
        end_date+=step
        dates=dates_all[start_date:end_date]
        #if input("Convert dates to UTC?(y/n)")=='y':
        #    dates=data.index.tz_localize('Europe/Moscow').tz_convert('utc')
        file_name=r'import data/METEOR-M 1.txt'#r'import data/'+input('Type the TLE filename(e.g. "glo-ops.txt", note: file must be in .txt extension, \n and plased in folder "import data"):  ') # type name of the file with TLE data
        satellite_name='METEOR-M 1'#input('Type satellite name(e.g. "COSMOS 2456"):  ') # previous file should contain TLE data for specifyed satellite
        with open(str(file_name), 'r') as TLE_data:
            all_data=TLE_data.readlines()
        for pos,line in enumerate(all_data):
            if line.find(satellite_name)!=-1:
                TLE_first_str=all_data[pos+1].strip() # read first line of TLE data
                TLE_sec_str=all_data[pos+2].strip()# # read second line of TLE data
                break
            elif pos==len(all_data): # if we finished end of the file and didn't find anythyng
                print('Couldn\'t find TLE')
        coord=[] # list for coordinates
        print("Running program: ")
        for procent,date in enumerate(dates): #procent using for progress bar
            print("\r{}%".format(int(ceil(procent*100/len(dates)))), end="") # progress bar
            #sys.stdout.flush() #<- makes python print it anyway, sometimes program shut down because of this line, so I commented her
            coord.append(state_vector(TLE_first_str, TLE_sec_str, date)) # obtain state vector for specifyed TLE and date
        coord= pd.DataFrame(coord, columns=['year','mon','day','hour','min','sec', 'x','y','z','altitude',  'latitude', 'longitude']) # transform list into more convenient DataFrame
        coord_for_igrf=coord.drop(['mon','day','hour','min','sec','x','y','z'], axis=1) # make pivot table for external programm that calculate L,B coordinates
        try:#try to delete temporal files, they are using by igrf.exe, if they doesn't exist it's OK
            os.remove('Path.txt')
            os.remove('Result.txt')
        except FileNotFoundError:
            print('')
        coord_for_igrf.to_csv('Path.txt', sep=' ', index=False,header=False, index_col=False) #make file with prepaired pivot table
        sleep(1)
        subprocess.run('igrf.exe') # this programm reads by defaul file 'Path.txt', so they must be in one folder. When finished, it creates resulting file "Result.txt". NOTE! igrf.exe won't create result file if there is one with the same name
        sleep(5)
        igrf_res=pd.read_csv('Result.txt', delim_whitespace=True)
        sleep(1)
        coord['L']=igrf_res['L-VALUE'] # append obtained l and B coordinates
        coord['B/B0']=igrf_res['B/B0']
        coord['epoch']=dates
        if len(dates)==1: # if in the beginning you entered just one date, then program prints it out on the screen
            print('\nRequested data by {}: \n \t state vector: x:{}, y:{}, z:{})\n \t longitude: {}\n \t latitude: {} \n \t L, B/B0: {}, {}'.format(str(coord[['year','mon','day','hour','min','sec']].ix[0].values),coord.x.ix[0],coord.y.ix[0],coord.z.ix[0],coord.longitude.ix[0],coord.latitude.ix[0], coord.L.ix[0],coord['B/B0'].ix[0]))
        if False:#input('\nsave satellite path to file? y/n: ')=='y': # try to save obtained data to .csv file wth separator ';'
            while True:
                try:
                    path=input('Type the file name: ')
                    coord.to_csv(r'export data/%s.csv' % path,sep=';', header=coord.columns,decimal=',',index_label=False, index=False)
                    break
                except OSError:
                    print('Name shouldn\'t contain \|/:*"<>?' )
        if True:#input('Plot picture of the satellite %s sub-path? y/n: '% satellite_name)=='y': # try to plot sub-path on the Earth map in Miller projection  
            plt.close('all')
            fig, ax=plt.subplots(nrows=1,ncols=1,figsize=(23.9,12.5))
            m=Basemap(ax=ax)
            m.drawcoastlines()
            m.drawcountries()
            m.etopo()
            x,y=m(coord.longitude,coord.latitude)
            if len(data)!=0 and len(data.columns)!=0:
                d_r=d_colored[:end_date]
                m_.set_array(d)
                clb=plt.colorbar(m_)
                #print(d)
                #print(len(x), len(y), len(d))
                path=m.scatter(x,y, c=d_r,marker=6,s=600, alpha=1)
                #clb=fig.colorbar(d[:end_date], ax=ax, shrink=0.761)
                clb.set_label('*{},counts/second'.format(round(pred[0].max(),2)))
            m.plot(x,y,'r+')
            ax.set_title('The satellite {} sub-path for the dates {}-{}'.format(satellite_name, dates[0], dates[-1]))
            if True:#input('Save obtained picture as png? y/n: ')=='y':
                plt.savefig(r'export data/gif/{}'.format(end_date))

def date_generator():
    global satellite_name
    global data
    global start_date 
    global end_date
    dates=[]
    while True:
        #dates_from_file=input('Dates from file? y/n: ')
        if True:#dates_from_file=='y':
            while True:
                try:
                    path=r'import data/METEOR_M1 2016-02-05--2016-04-06.csv'#+input('Type filename(e.g. "dates.csv"(without quotes); note: separator=";",decimal=".", dates must be in UTC, \n and plased in time_to_LB.ipynb directory in folder "input data"): ')
                    print("Loading file:")
                    data= pd.read_csv(path, sep=';',decimal=',', header=None, parse_dates=[0],dayfirst=True, infer_datetime_format=True, index_col=0)
                    print("Succeed!")
                    break
                except OSError:
                    print('Invalid filename, try again')
            if True:#input('All dates from file? y/n: ')=='n':
                start_date='2016-04-05 06:00:00'#input('Input the date of the beginning in the format: "YYYY-MM-DD hh:mm:ss":  ')
                end_date='2016-04-05 14:48:00'#input('Input the date of the end in the format: "YYYY-MM-DD hh:mm:ss":  ')
                data=data[start_date:end_date]
            else:
                start_date=data.index[0]
                end_date=data.index[-1]
            return data.index
            break
        elif dates_from_file=='n':
            date_or_range=input('Certain date or range? y - certain date, n - range: \n')
            if date_or_range=='n':
                start_date='2016-04-05 06:00:00'#input('Input the date of the beginning in the format: "Y/M/D h:m:s":  ')
                end_date=input('Input the date of the end in the format: "Y/M/D h:m:s":  ')
                freq=input('Input the frequensy in the format: %digit%string % [1,2,3...],["s","min","h","d","m","y"]\n(e.g. "10min" or "1s"): ' )
                dates=pd.date_range(start_date,end_date,freq=freq)
                return dates
                break
            elif date_or_range=='y':
                if input('current date?(Important! your local time will be recalculated into UTC time) y - current date, n - other date: \n')=='y':
                    dates=[datetime.utcnow()]
                    return dates
                    break
                dates=[parse(input('Input the date in the format: "Y/M/D h:m:s":  '))]
                return dates
                break
            else:
                print('Wrong input, try again')
                continue
        else:
            print('Wrong input, try again')
            continue
def star_time(date):
    #d_s=[int(x.strip()) for x in date.split(sep=',')]
    year=date.year#d_s[0]
    mon=date.month#d_s[1]
    day=date.day#d_s[2]
    hour=date.hour#d_s[3]
    minute=date.minute#d_s[4]
    sec=date.second#d_s[5]
    a=math.trunc((14-mon)/12)
    y=year+4800-a
    m=mon+12*a-3
    JDN=day+math.trunc((153*m+2)/5)+365*y+math.trunc(y/4)-math.trunc(y/100)+math.trunc(y/400)-32045
    JD=JDN+(hour-12)/24+minute/1440+sec/86400
    MJD=JD-2400000.5 # modifyed Julian date
    # star time 
    MJD0=math.trunc(MJD)
    UT=(MJD-MJD0)*24
    T=(MJD0-51544.5)/36525.0
    GMST=6.697374558+1.0027379093*UT+(8640184.812866+(0.093104-(6.2e-6*T))*T)*T/3600.0
    def FRAC(X):
        X=X-math.trunc(X)
        if X<0:
            X+=1
        return X
    LMST=24.0*FRAC(GMST/24.0)
    #hour_LMST=math.trunc(LMST)
    #minute_LMST=math.trunc((LMST-math.trunc(LMST))*60)
    #sec_LMST=math.trunc(((LMST-math.trunc(LMST))*60-math.trunc((LMST-math.trunc(LMST))*60))*60)
    return LMST
def state_vector(TLE_first_str, TLE_sec_str, date):
    '''
    This function recuires imported sgp4.earth_gravity.wgs72 and sgp4.io.twoline2rv.
    It brings state vector(coordinates in ECI(in km), and velocity for each axe(in km/h)) for required satellite
    
    Inputs: (file_name, sat_name, data)
            file_name - name of text doc in .txt, where are all TLE data agregated, file_name should ends with '.txt', ex:'glo-ops.txt'
            sat_name - satellite name, specifyed in doc file_name, ex.: 'COSMOS 2419' in file 'glo-ops.txt'
            date - date in string, format: 'YYYY, MM, DD, HH, mm, SS', if didgit of the day, hour, etc. can be written by one digit, then you should write it only in this way, without zeros, ex.:'2016, 9, 29, 13, 56, 59', here month - '9'. NOT '09'!!! 
    Outputs: function returns tuple with state vector and prints extracted TLE data, and date, specifyed in input
    '''
    year=date.year#d_s[0]
    mon=date.month#d_s[1]
    day=date.day#d_s[2]
    hour=date.hour#d_s[3]
    minute=date.minute#d_s[4]
    sec=date.second#d_s[5]
    # obtain state vector in ECI:
    satellite=twoline2rv(TLE_first_str, TLE_sec_str, wgs72)
    position,velocity=satellite.propagate(year, mon, day, hour, minute, sec) 
    x=position[0]
    y=position[1]
    z=position[2]
    # transit from ECI to longitude, latitude in inertial coordiantes
    longitude=math.atan(y/x)*180/math.pi
    sign=lambda x: int(math.copysign(1,x))
    if (sign(y)>0 and sign(x)<0) or (sign(y)<0 and sign(x)<0):
        longitude+=180
    if sign(y)<0 and sign(x)>0:
        longitude+=360
    r=math.sqrt(x**2+y**2)
    lat=math.atan(z/r)
    # calculate star time for Gregorian calendar
    LMST=star_time(date)
    delta_longitude=7.2921158553*10**(-5)*LMST*3600*180/math.pi
    # transit from polar inetial system to polar relative system
    long_rel=longitude-delta_longitude
    if long_rel>180:
        long_rel-=360
    elif long_rel<-180:
        long_rel+=360
    #recalculate lat, alt for nonspherical Earth:
    a=6378.135 #equatorial Earth's radius
    f=1/298.26 #Earth's shrinkage
    ecc=2*f-pow(f,2) #square of Earth's eccentricity
    while True:
        lat_=lat
        C=1/sqrt(1-ecc*pow(sin(lat_),2))
        lat=math.atan((z+a*C*ecc*math.sin(lat_))/math.sqrt(x**2+y**2))
        if abs(lat_-lat)<0.0001:
            break
    alt=math.sqrt(x**2+y**2)/cos(lat)-a*C
    lat=lat*180/math.pi
    return year, mon, day,hour,minute,sec, x,y,z, alt, lat, long_rel


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy
